In [4]:
import mne
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader,Subset
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import json

In [11]:
mne.set_log_level('ERROR')

In [47]:
class SeizureSense(nn.Module):
    
    def __init__(self):
        super(SeizureSense,self).__init__()
        
        #Block 1
        #should be taking in an input of 23x512
        #first layer temporal shittt(pointwise is the technical term)
        self.conv1=nn.Conv2d(1,8,(1,128),stride=1,padding=0)
        self.batchnorm1 = nn.BatchNorm2d(8, False)
        
        #spatial layer(depthwise layer)
        self.conv2=nn.Conv2d(8,32,(23,1))
        self.batchnorm2 = nn.BatchNorm2d(32, False)
        self.avgpool1 = nn.AvgPool2d((1,2))
        #apply dropout here in forward
        
        
        #Block 2
        #sepereable convolutional 2d
        self.conv3=nn.Conv2d(32,32,(1,16),stride=1)
        self.batchnorm3 = nn.BatchNorm2d(32,False) #CHECK THIS!
        self.avgpool2 = nn.AvgPool2d((1,16))
        #apply dropout here in forward
        
        
        
        #lstm layers
        
        #Block 3
        #fc1 layer assumiing no bilstm layer rn can change later 
        #matthew check my math here please its 2am 
        
        self.fc1= nn.Linear(96, 30)
        self.fc2=nn.Linear(30,3)
        
        
        
    def forward(self,x):
        x=self.conv1(x)
        x =F.elu(x)
        x=self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        
        x = self.conv2(x)
        x = F.elu(x)
        x=self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x=self.avgpool1(x)
        
        x=F.elu(self.conv3(x))
        x=self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x=self.avgpool2(x)
        
        #bilstm stufff
        
        #Fully connected time
        x=x.view(-1,96)
        x=F.elu(self.fc1(x))
        x=F.sigmoid(self.fc2(x))
        return x

In [9]:
model=SeizureSense()
state_dict = torch.load(r"C:\Users\uddha\Downloads\SeizureSenseStateDict.pth",map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
#loading data
raw = mne.io.read_raw_edf(r"C:\Users\uddha\Downloads\chb02_19.edf")
raw.load_data()
#Interictal time steps
#2000 seceonn-2001 seceonds
#Preictal 
#2779 seceonds-2780 seceonds
#inseceonds to 3371 seceonds

In [48]:
numpy_array=raw.get_data()
segment_tensor=torch.from_numpy(numpy_array).float().unsqueeze(0).unsqueeze(0)
print(segment_tensor.shape)

torch.Size([1, 1, 23, 921600])


In [49]:

#testing on different inputs: no preprocessing
input=segment_tensor[:,:,:,256*3370:256*3371]

output=model(input)
predicted_label_names = decode_predictions(output)
print(predicted_label_names)

['interictal']


In [ ]:
#maps back the labels to words
def decode_predictions(predictions):
    label_names = ['interictal', 'preictal', 'ictal']
    _, predicted_labels = torch.max(predictions, 1)
    return [label_names[label] for label in predicted_labels]

predicted_label_names = decode_predictions(output)
print(predicted_label_names)

In [34]:
#testing with preprocessing
#filtering 60 hz out with notch filtering

def preprocessing(raw):
    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)


    freqs = (60,120)
    raw_notch = raw.copy().notch_filter(freqs=freqs, picks=eeg_picks)
    raw_notch.filter(l_freq=30, h_freq=None, fir_design='firwin', filter_length='auto', phase='zero', fir_window='hamming')
    numpy_array=raw_notch.get_data()
    segment_tensor=torch.from_numpy(numpy_array).float().unsqueeze(0).unsqueeze(0)
    return segment_tensor

In [38]:
segment_tensor=preprocessing(raw)

In [46]:
input=segment_tensor[:,:,:,256*2000:256*2001]
output=model(input)
predicted_label_names = decode_predictions(output)
print(predicted_label_names)

['interictal']


In [83]:
#testung model1 
model1=SeizureSense()
#without preprocessing
input=segment_tensor[:,:,:,256*2000:256*2001]
output=model1(input)
predicted_label_names = decode_predictions(output)
print(predicted_label_names)

['preictal']
